In [ ]:
# Welcome!

    The first cell here contains some of the functions and modules that we used in our code. The 'kaggle' module is NOT needed, as we only used it to download the API from the kaggle website for our datasets. The other modules are needed however, as they are integral to ensuring the code functions properly.

In [189]:
import csv
import kaggle
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import re

    The code below reads in 2 CSV files. They both contain common abbreviations in the english language AND their expanded forms. They are already arranged in a hashmap type of format, so we have created two hashmaps to represent each file, and combined them together to create one big dictionary to use. If you need to add any more values, there are examples of adding a couple of abbreviations to the dictionary.

In [190]:
with open('Abbreviations and Slang.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}

with open('contractions.csv', mode = 'r') as infile:
    reader = csv.reader(infile)
    mydict1 = {rows[0]:rows[1] for rows in reader}

dictionaryFinal = mydict | mydict1

dictionaryFinal["u"] = 'you'
dictionaryFinal["btc"] = 'bitcoin'
dictionaryFinal["eth"] = 'ethereum'
dictionaryFinal["usd"] = 'us dollar'

    The code below reads in the dataset.csv file, which contains all of the tweets that are being iterated over and used for this code. 

In [191]:
df = pd.read_csv("dataset.csv", dtype = {"Text": str, "Language": str, "Label": str})
len(df)

937854

    The below code is the implementation of the first Algorithm.
 
    There are functions to measure the time complexity of the first algorithm and print them out. To adjust the amount of data used, the FOR loop uses a range to determine how many tweets to iterate over. 

In [192]:
start = timer()
for i in range(0, 1000):
    string2 = str(df["Text"][i])
    pattern = r"(\b\w{1,6}[a-z]\b)|(\b\w+[’']\w+\b)"
    x = re.finditer(pattern, string2)
    for match in x:
        # print(match.start(), match.group(), match.end())
        if(match.group() in dictionaryFinal):
            startOfAb = match.start()
            abValue = match.group()
            expandedAb = dictionaryFinal.get(abValue)
            # print("It matches!")
            # def replace() -> algorithm for replacing the word in the tweet.
end = timer()
print(end - start)

0.01732570000058331


    The code below is the implementation of the second algorithm. We decided to define it as a separate function to create less confusion in the main cell. It iterates through ONE tweet at a time, recursively. The recursion occurs when there is more than one abbreviation in a single tweet. It will replace the abbreviations one by one, using the new tweets as input.

In [193]:
def replaceAb(tweet, matches, indices, expandedForms, i, count):
    done = False
    newTweet = tweet.replace(matches[i], expandedForms[i])
    count = count + 1
    while(done != True):
        if(count >= len(matches)):
            done = True
            return newTweet
        else:
            return replaceAb(newTweet, matches, indices, expandedForms, i + 1, count)
            done = True

# This code is the main cell.
    It contains the code for the entire program, in one place. The commented cells are only used to measure the time complexity of the SECOND algorithm. As such, they are not needed for the main functionality of the code. 

In [199]:
#sumTimer = 0
#startOfLoop = 0
#endOfLoop = 0
#count = 0
newTweet = ""
for i in range(0, 1000):
    string2 = str(df["Text"][i])
    pattern = r"(\b\w{1,6}[a-z]\b)|(\b\w+[’']\w+\b)"
    x = re.finditer(pattern, string2)

    matches = []
    indices = []
    expandedForms = []

    for match in x:
        if(match.group() in dictionaryFinal):
            matches.append(match.group())
            indices.append(match.start())
            expandedForms.append(dictionaryFinal.get(match.group()))
            #count = count + 1
        else:
            continue
    
    
    if(len(matches) > 0):
        #startOfLoop = timer()
        newTweet = replaceAb(string2, matches, indices, expandedForms, 0, 0)
        #endOfLoop = timer()
    else:
        continue
    print("Original tweet:", string2, "\n\nNew Tweet:", newTweet, "\n")
    #sumTimer = sumTimer + (endOfLoop - startOfLoop)
#print(sumTimer, count)


Original tweet: It sucks for me since I'm focused on the nature aspect of things but the environmental problems we have all have their roots in social, economic &amp; political issues. Simply recycling or using renewable energy is like fixing the top floor of a building with an unsteady foundation 

New Tweet: It sucks for me since I am focused on the nature aspect of things but the environmental problems we have all have their roots in social, economic &amp; political issues. Simply recycling or using renewable energy is like fixing the top floor of a building with an unsteady foundation 

Original tweet: @roipaee @joe98100959 @Formula1game @redbullracing @SilverstoneUK I wrote to them now,waiting for a response...I've lost hope
With FIFA games worked every year 

New Tweet: @roipaee @joe98100959 @Formula1game @redbullracing @SilverstoneUK I wrote to them now,waiting for a response...I have lost hope
With FIFA games worked every year 

Original tweet: @JaneAnne_E I've probably written